### Carga de las imagenes

In [1]:
from image_services import get_image_urls
from skimage import io
import solar_image_analysis as sia
import re 
from datetime import datetime

Acá esta como ejemplo como podemos generar todos los links para cualquera de las  carpertas dado un rango de fechas, el nombre de la carpeta (tipo de imagen, en que banda se tomo), indicando apropiadamente la fecha de incio y la frecuencia con la que se toman, como se muestra a continuación estan generando para tres casos


In [12]:
# Metodo 1
hmiigr_urls = get_image_urls( image_type = 'hmiigr',start='01/01/2022 00:00:00', end= '01/01/2023', frequency='90T')

# Metodo 2
mdimag_urls =  get_image_urls( image_type = 'mdimag', year= '1997')


Acá se estan probando el calculo de los parametros que se van implementando para 10 imagenes

In [20]:
import pandas as pd
from urllib.error import HTTPError

def compute_metrics(image_type:str, year:str = None, start:str = None, end:str = None,  frequency:str=  None ) -> pd.DataFrame:
    
    """
    Computes various image metrics for a set of images within a given date range and specified type and frequency.

    Parameters
    ----------
    start_date : str
        Start date for retrieving the images, formatted as 'YYYY-MM-DD'.
    
    end_date : str
        End date for retrieving the images, formatted as 'YYYY-MM-DD'.
    
    image_type : str
        Type/category of the image to retrieve. 
    
    frequency : str
        Frequency for retrieving images, e.g., 'daily', 'hourly'.

    Returns
    -------
    pd.DataFrame
        DataFrame containing computed metrics for each image, with one row per image 
        and columns for each feature. The index of the DataFrame corresponds to the datetime 
        extracted from the image URL.
        
    Notes
    -----
    The function fetches image URLs based on the provided date range, type, and frequency. 
    For each URL, it reads the image and computes a set of metrics, such as entropy, mean 
    intensity, and fractal dimension. If the image at a particular URL is inaccessible, it 
    skips that URL. The extracted date and time from the image URL is used as the index 
    for the resulting DataFrame.
    """

    image_urls = get_image_urls(image_type, year, start, end,  frequency )
    
    #Create an empty DataFrame to store the results
    df = pd.DataFrame(columns=['entropy', 'mean_intensity', 'std_deviation', 'fractal_dimension', 'skewness', 'kurtosis', 'uniformity', 'relative_smoothness', 
                                'taruma_contrast', 'taruma_directionality'])

    for url in image_urls:
        try:
            # Try to read the image
            image = io.imread(url)
            

        except HTTPError:
            #print(f"Unable to access image at {url}")
            continue

        if len(image.shape) == 3:
            
            # Compute the features
            entropy = sia.compute_entropy(image)
            mean_intensity = sia.compute_mean_intensity(image)
            std_deviation = sia.compute_standard_deviation(image)
            fractal_dimension = sia.compute_fractal_dimension(image)
            skewness = sia.compute_skewness(image)
            kurtosis = sia.compute_kurtosis(image)
            uniformity = sia.compute_uniformity(image)
            relative_smoothness = sia.compute_relative_smoothness(image)
            taruma_contrast = sia.compute_taruma_contrast(image)
            taruma_directionality = sia.compute_taruma_directionality(image)
            # Extract the date from the URL
            date_string = re.findall(r"\d{8}_\d{4}", url)[0]
            date = datetime.strptime(date_string, '%Y%m%d_%H%M')

            # Append the results to the DataFrame
            df.loc[date] = [entropy, mean_intensity, std_deviation, fractal_dimension, skewness, kurtosis, uniformity, relative_smoothness,taruma_contrast, taruma_directionality ]

        else: 
            pass

    return df

# Este es el metodo que teniamos solo con una modificación en el orden de los parametros
#data = compute_metrics( image_type = 'hmiigr',start='01/01/2022 00:00:00', end= '01/01/2023', frequency='90T')
    
# Esta es la otra opción, para años anteriores a 2011
data = compute_metrics( image_type = 'mdimag', year= '1997')
#print(data)


,entropy,mean_intensity,std_deviation,fractal_dimension,skewness,kurtosis,uniformity,relative_smoothness,taruma_contrast,taruma_directionality
2023-01-01 01:06:00,6.530609,55.195968,47.971115,1.404219,0.959536,4.220139,0.044851,0.999566,1605.565149,0.980127
2023-01-01 13:06:00,6.494467,55.083712,47.767550,1.388460,0.885746,4.000265,0.048005,0.999562,1613.406314,0.981164
2023-01-02 01:06:00,6.518386,55.862615,47.180088,1.392160,0.772212,3.699534,0.045877,0.999551,1605.021067,0.980525
2023-01-02 13:06:00,6.514073,56.257136,47.180465,1.406327,0.707689,3.510225,0.046295,0.999551,1626.262618,0.980570
2023-01-03 01:06:00,6.504687,55.925506,46.903886,1.406577,0.683620,3.426778,0.046660,0.999546,1616.948354,0.980645
2023-01-03 13:06:00,6.526086,56.860770,47.356625,1.419623,0.691289,3.570479,0.044945,0.999554,1631.472968,0.979879
2023-01-04 01:06:00,6.550122,58.072339,47.142782,1.417613,0.653803,3.591307,0.042222,0.999550,1614.422754,0.978860
2023-01-04 13:06:00,6.562257,58.606423,47.296687,1.418816,0.659711,3.669546,0.041215,0.999553,1616.249344,0.978448
2023-01-05 01:06:00,6.568673,59.058887,47.289471,1.423777,0.655731,3.730987,0.040290,0.999553,1609.062764,0.866685
2023-01-05 13:06:00,6.608578,59.625464,47.941407,1.426468,0.698014,3.830403,0.038731,0.999565,1642.897350,0.655895
